# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [19]:
using Manifolds
using Manopt
using LinearAlgebra
using NIfTI
using Plots
using LaTeXStrings
using JLD

In [20]:
include("../../../src/decompositions/tensors/naive_low_multilinear_rank_approximation.jl")
include("../../../src/decompositions/tensors/curvature_corrected_low_multilinear_rank_approximation.jl")
include("../../../src/decompositions/tensors/iterative_curvature_corrected_low_multilinear_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")

exact_loss (generic function with 2 methods)

### Load data and construct manifold ###

In [21]:
# load data
ni = niread("../../../data/nifti_dt_nonlinear.nii.gz") * 1e9;
size(ni)

(112, 112, 50, 1, 6)

In [22]:
 d1, d2, d3, _, d = size(ni)

# construct data as points on the manifold (and add 1e-5 * I for numerical stability)
predata = [ # data ordered as [xx, yx, yy, zx, zy, zz]
    [
    [ni[i,j,k,1,1] + 1e-5;; ni[i,j,k,1,2];; ni[i,j,k,1,4]]; 
    [ni[i,j,k,1,2];; ni[i,j,k,1,3] + 1e-5;; ni[i,j,k,1,5]]; 
    [ni[i,j,k,1,4];; ni[i,j,k,1,5];; ni[i,j,k,1,6] + 1e-5]
    ] for i=1:d1, j=1:d2, k=1:d3];
    
# pick a 2D slice 
data = predata[38:77,36:75,15];
# data = data[1:10,1:10]
println("data size = $(size(data))")

# construct data manifold
n = size(data)
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M) 

data size = (40, 40)


6

In [23]:
# Export slice image
asymptote_export_SPD("results/Camino2D_orig.asy", data=data);

### Construct low rank approximation ###

In [24]:
unrolled_data = []
for mat in data
    push!(unrolled_data,mat)
end
q = mean(M, unrolled_data)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [25]:
curvature_corrected_low_multilinear_rank_approximation(M, q, data, [5,5])

([[0.8950044738973227 0.6692007023653097 0.18192408878533542; 0.6692007023653097 -0.3959969445677654 -0.1943966809004528; 0.18192408878533542 -0.1943966809004528 0.520395048654941] [4.778954643848296 0.8350167502433133 -0.6096050134116588; 0.8350167502433133 3.4546565459092684 -0.29378223771945977; -0.6096050134116588 -0.29378223771945977 3.591810436594278] … [-1.5789986320626614 1.818457276265771 0.5161712206847788; 1.818457276265771 -1.0787680982280126 -0.6785620671386691; 0.5161712206847788 -0.6785620671386691 -2.029295990487366] [1.2666231856002175 -1.3857754891774232 0.7759527838512013; -1.3857754891774232 0.8360238590419994 -0.5417704520633215; 0.7759527838512013 -0.5417704520633215 -1.1620456970187312]; [-3.4061642776829895 0.6555409784628693 -0.2285094606598303; 0.6555409784628693 -3.4678282342077376 1.0460681109163212; -0.2285094606598303 1.0460681109163212 -4.936097465547379] [2.3640118956035483 1.4383978171194534 -1.0177369726704752; 1.4383978171194534 0.6779202101827806 -0.

In [26]:
# rank_range = collect(1:min(n[1],n[2]))
# rank_range = collect(1:3:20)
rank_range = collect(1:3:min(n[1],n[2]))

ranks = []
nR_q = []
nU = []
ccR_q = []
ccU = []

for rank in rank_range
    push!(ranks,rank)
    println("#$(rank) | computing naive low-rank approximation")
    nRr_q, nUr = naive_low_multilinear_rank_approximation(M, q, data, [rank,rank]); 
    push!(nR_q, nRr_q)
    push!(nU, nUr)
    println("#$(rank) | computing curvature corrected low-rank approximation")
    ccRr_q, ccUr = curvature_corrected_low_multilinear_rank_approximation(M, q, data, [rank,rank]); 
    push!(ccR_q, ccRr_q)
    push!(ccU, ccUr)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
#4 | computing naive low-rank approximation
#4 | computing curvature corrected low-rank approximation
#7 | computing naive low-rank approximation
#7 | computing curvature corrected low-rank approximation
#10 | computing naive low-rank approximation
#10 | computing curvature corrected low-rank approximation
#13 | computing naive low-rank approximation
#13 | computing curvature corrected low-rank approximation
#16 | computing naive low-rank approximation
#16 | computing curvature corrected low-rank approximation
#19 | computing naive low-rank approximation
#19 | computing curvature corrected low-rank approximation
#22 | computing naive low-rank approximation
#22 | computing curvature corrected low-rank approximation
#25 | computing naive low-rank approximation
#25 | computing curvature corrected low-rank approximation
#28 | computing naive low-rank approximation
#28 | computing curvature

In [27]:
num_ranks = length(ccU)
ref_distance = sum(distance.(Ref(M), Ref(q), data).^2)

naive_tangent_distances_r = zeros(num_ranks)
predicted_naive_distances_r= zeros(num_ranks)
true_naive_distances_r= zeros(num_ranks)

curvature_corrected_tangent_distances_r = zeros(num_ranks)
predicted_curvature_corrected_distances_r = zeros(num_ranks)
true_curvature_corrected_distances_r = zeros(num_ranks)

for (idx,rank) in enumerate(ranks)
    naive_log_q_data_r = Symmetric.([sum([nR_q[idx][i,j] * nU[idx][1][k,i] * nU[idx][2][l,j] for i in 1:rank, j in 1:rank]) for k=1:n[1], l=1:n[2]])
    curvature_corrected_log_q_data_r = Symmetric.([sum([ccR_q[idx][i,j] * ccU[idx][1][k,i] * ccU[idx][2][l,j] for i in 1:rank, j in 1:rank]) for k=1:n[1], l=1:n[2]])
    # expoentiate back
    naive_data_r = exp.(Ref(M), Ref(q), naive_log_q_data_r)
    curvature_corrected_data_r = exp.(Ref(M), Ref(q), curvature_corrected_log_q_data_r)

    # compute relative tangent space error
    naive_tangent_distances_r[idx] = sum(norm.(Ref(M), Ref(q),  log_q_data - naive_log_q_data_r).^2) / ref_distance
    curvature_corrected_tangent_distances_r[idx] = sum(norm.(Ref(M), Ref(q),  log_q_data - curvature_corrected_log_q_data_r).^2) / ref_distance

    # compute relative manifold error
    predicted_naive_distances_r[idx] = curvature_corrected_loss(M, q, data, naive_log_q_data_r)
    true_naive_distances_r[idx] = exact_loss(M, q, data, naive_log_q_data_r)
    predicted_curvature_corrected_distances_r[idx] = curvature_corrected_loss(M, q, data, curvature_corrected_log_q_data_r)
    true_curvature_corrected_distances_r[idx] = exact_loss(M, q, data, curvature_corrected_log_q_data_r)
end

In [28]:
plot(ranks, [naive_tangent_distances_r, true_naive_distances_r, true_curvature_corrected_distances_r], label = ["theoretical lower bound" "naive" "curvature corrected"], ylims=(0,1), xlims=(1,max(ranks...)),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/Camino2D_errors_by_rank.png")
plot(ranks, [naive_tangent_distances_r .+ 1e-16, true_naive_distances_r .+ 1e-16, true_curvature_corrected_distances_r .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected"], ylims=(1e-16,1), xlims=(1,max(ranks...)), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/Camino2D_logerrors_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/2D/P3/results/Camino2D_logerrors_by_rank.png"

In [29]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
plot(ranks, [predicted_naive_distances_r ./ true_naive_distances_r, predicted_curvature_corrected_distances_r ./ true_curvature_corrected_distances_r], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,max(ranks...)),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/Camino2D_discrepancy_by_rank.png")
plot(ranks, [predicted_naive_distances_r ./ true_naive_distances_r, predicted_curvature_corrected_distances_r ./ true_curvature_corrected_distances_r], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,max(ranks...)), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
savefig("results/Camino2D_logdiscrepancy_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/2D/P3/results/Camino2D_logdiscrepancy_by_rank.png"